# Ejercicio 1.1

In [ ]:
import numpy as np
from numpy.fft import fft, fftfreq, fftshift

t = np.linspace(0, 10e-9, num = 1001)
dt = t[1] - t[0]
t_0 = 4e-9
s_0 = 1e-9
f_t = np.exp(-pow(t-t_0,2)/(2*s_0**2))
#f_t = 2*np.sin(2*np.pi*t)

import matplotlib.pyplot as plt

plt.figure()
plt.plot(t,f_t, 'b')
plt.grid()

plt.figure()
freq = fftshift( fftfreq(len(f_t), d = dt) ) # fftshift reordena las frecuencias (de forma general fft las devuelve al revés)
fdf_t = fftshift( fft(f_t) )

plt.plot(freq, np.abs(fdf_t), 'b')
plt.grid()

Vamos a generar animaciones de una gaussiana que se desplaza hacia una dirección. Se ha hecho en Animation.py

# Esquema fdt de las ecuaciones de Maxwell

Las ecuaciones de Maxwell en una dimenesión se pueden expresar como:

\begin{equation}
    \mu\partial_t H + \partial_x E = 0
    \nonumber
\end{equation}

\begin{equation}
    \varepsilon\partial_t E + \partial_x H = 0
    \nonumber
\end{equation}

Discretizando las ecuaciones usando la aproximación central, obtenemos la siguiente ecuación

\begin{equation}
    H^{n+\frac{1}{2}}_{i+\frac{1}{2}} = H^{n-\frac{1}{2}}_{i+\frac{1}{2}} + \frac{\Delta t}{\Delta x\mu}\left( E^{n}_{i+1} - E^{n}_{i-1}\right)
    \nonumber
\end{equation}

\begin{equation}
    E^{n}_{i} = E^{n-1}_{i} + \frac{\Delta t}{\Delta x \varepsilon}\left( H^{n-\frac{1}{2}}_{i+\frac{1}{2}} - H^{n-\frac{1}{2}}_{i-\frac{1}{2}}\right)
    \nonumber
\end{equation}

Definimos primero las constantes del problema

In [ ]:
import numpy as np

eps = 1
mu = 1
c = 1/np.sqrt(mu*eps)

Ahora, construimos el grid primario (en el que vive E) y el secundario (En el que vive H)

In [ ]:
N = 101

x_m = np.linspace(0, 10, N) # grid primario (main)
x_d = (x_m[1:] + x_m[:-1])/2 # En esta construcción es claro que el campo magnético tiene un nodo menos que el eléctrico

dx = x_m[1] - x_m[0] # Definimos el paso espacial


In [ ]:
x_0 = 3
s_0 = 0.75

E = np.exp(-pow(x_m-x_0,2)/(2*s_0**2))
H = np.zeros(x_d.shape) 
E_new = np.zeros(E.shape)
H_new = np.zeros(H.shape)

import matplotlib.pyplot as plt

plt.figure()
plt.plot(x_m, E, 'b.-')
plt.plot(x_d, H, 'r.-')
plt.title('Condición inicial de $E(t,x)$ y $H(t,x)$')
plt.xlabel('x (m)')
plt.ylabel('$E(t,x)$ $H(t,x)$')
plt.grid()

Evolucionamos el sistema

In [ ]:
CFL = 0.4 # condición CFL de estabilidad

dt = CFL * dx / c

t_range = np.arange(0, 2*dt, dt)

for t in t_range:

    E_new[1:-1] = E[1:-1] - (dt / dx / eps)*( H[1:] - H[:-1] )
    E[1:-1] = E_new[1:-1]
    H_new[:]    = H[:]    - (dt / dx / mu) *( E_new[1:] - E_new[:-1] )
    H[:] = H_new[:]

    plt.plot(x_m, E, 'b.-')
    plt.plot(x_d, H, 'r.-')
    plt.grid()
    plt.ylim(-0.1, 1.1)
    plt.xlim(x_m[0], x_m[-1])
    plt.pause(0.1)
    plt.cla()

Vamos a estudiar ahora algunas variaciones del problema. Veremos qué ocurre al cambiar $\varepsilon$, $\mu$ y $\sigma$.

Las ecuaciones de Maxwell en una dimensión en el caso de ser un medio conductor:

\begin{equation}
    \mu\partial_t H + \partial_x E = 0
    \nonumber
\end{equation}

\begin{equation}
    \sigma E + \varepsilon\partial_t E + \partial_x H = 0
\end{equation}

Si discretizamos la ecuación $(1)$ obtenemos la ecuación:

\begin{equation}
    \sigma_i E^{n-\frac{1}{2}}_i + \varepsilon_i \frac{E^{n}_{i} - E^{n-1}_{i}}{\Delta t} = -\frac{ H^{n+\frac{1}{2}}_{i+\frac{1}{2}} - H^{n+\frac{1}{2}}_{i-\frac{1}{2}}}{\Delta x}
    \nonumber
\end{equation}

Podemos aplicar el operador media al término $E^{n-\frac{1}{2}}_i$ ya que así obtendríamos el cmapo en función de su valor en $n-1$ y $n$.

\begin{equation}
    E^{n-\frac{1}{2}}_i = \frac{E^{n}_i+E^{n-1}_i}{2}
    \nonumber
\end{equation}

Finalmente despejamos el sistema y obtenemos la siguiente ecuación:

\begin{equation}
    E^{n}_i = -\frac{\alpha}{\beta}E^{n-1}_i-\frac{1}{\Delta x \alpha}\left( H^{n+\frac{1}{2}}_{i+\frac{1}{2}} - H^{n+\frac{1}{2}}_{i-\frac{1}{2}} \right) 
    \nonumber
\end{equation}

Donde $\alpha$ y $\beta$ son

\begin{equation}
    \alpha = \left(\frac{\sigma_i}{2} + \frac{\varepsilon_i}{\Delta t} \right), \beta = \left(\frac{\sigma_i}{2} - \frac{\varepsilon_i}{\Delta t} \right)
\end{equation}


Si introducimos la condición incial para el campo magnético y eléctrico, observamos una pequeña onda hacia detrás. esto ocurre porque el campo magnético vive en un paso $\frac{\Delta t}{2}$ anterior al eléctrico.

# Condiciones de contorno

Las condiciones que hemos usado hasta ahora son las de `un conductor electrico perfecto` `(PEC)` en el contorno. Lo que hacemos es suponer que en el medio 2 el campo eléctrico $\vec{E_2}$ es cero es 0.

\begin{equation}
    \hat{n} \times (\vec{E_2}-\vec{E_1}) = 0 \rightarrow \hat{n} \times \vec{E} = 0
    \nonumber
\end{equation}

En el esquema fdtd lo que hacemos simplemente es fijar en los extremos del intervalo el valor del campo eléctrico a 0.

\begin{equation}
    E_{0}^{n+1} = 0
    \nonumber
\end{equation}

Otra condición posible sería la de un `un conductor magnético perfecto` `(PMC)` en el contorno. Aunque no existe en la realidad es útil para reducir la superficie de simulación usando también el PEC (Se puede ver usando la idea del método de las imágenes).

\begin{equation}
    \hat{n} \times (\vec{H_2}-\vec{H_1}) = \vec{J} \rightarrow \hat{n} \times \vec{H} = 0
    \nonumber
\end{equation}

Para implementar esta condición en el esquema fdtd lo que hacemos es partir de la Ley de Ampere discretizada y decir que el valor $H_{-\frac{1}{2}}^{n+\frac{1}{2}}$ (que no existe) vale lo mismo que $H_{+\frac{1}{2}}^{n+\frac{1}{2}}$ para uqe al hacer la aproximación de la media el campo en el contorno nos de 0 por lo que la condición que debe cumplirse es

\begin{equation}
    E_{i = 0}^{n+1} = 0...
    \nonumber
\end{equation}

Otra condición sería una `absorvente`, esta codnción se conoce como `condición absorvente de Mur`. Consiste en discretizar en $i = \frac{1}{2}$ y de imponer la condición de que las ondas no se reflejan

\begin{equation}
    (\partial_n + c_0^{-1} \partial_t)(\hat{n} \times \vec{E}) = 0
    \nonumber
\end{equation}

Que al discretizarlo en $x = 0$ se traduce en

\begin{equation}
    \frac{ E^{n+\frac{1}{2}}_{1} - E^{n+\frac{1}{2}}_{0}}{\Delta x} + c_0^{-1} \frac{E^{n}_{\frac{1}{2}} - E^{n-1}_{\frac{1}{2}}}{\Delta t}=0
    \nonumber
\end{equation}

# Error asociado al esquema fdtd

Si $f(v)$ es una función harmónica de la forma $Acos(2\pi/\lambda_v v)$ el error comentido en la aproximación central de la derivada 
y en la aproximación de la media de $f(v)$ viene dada por

\begin{equation}
    \Delta f_v = \frac{\pi^2}{6}\frac{1}{\lambda_v^2} \left(\Delta v\right)^2
    \nonumber
\end{equation}

\begin{equation}
    \Delta E\left[f \right] = 3\Delta f_v = \frac{\pi^2}{2}\frac{1}{\lambda_v^2} \left(\Delta v\right)^2
    \nonumber
\end{equation}

# Relación de dispersión

Escribimos las ecuaciones de maxwell como:

\begin{equation}
    \left(\Lambda_t-\Lambda_r\right)\psi = 0
\end{equation}

con

\begin{matrix}
    \Lambda_t = \left( \right)
\end{matrix}
Podemos discretizar $\psi = \psi_{i,j,k}^n$